<a href="https://colab.research.google.com/github/eli-osherovich/mnist-demo/blob/main/mnist_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist, fashion_mnist

In [2]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
# Set training context (TPU, GPU or CPU)
try:
  tpu =  tf.distribute.cluster_resolver.TPUClusterResolver()
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])

  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)

  strategy = tf.distribute.TPUStrategy(tpu)
  # policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
except ValueError:
  print("Cannot conect to tpu, using default accelerator")
  strategy = tf.distribute.get_strategy()
  # policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')



Running on TPU  ['10.72.115.82:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.72.115.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.72.115.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [4]:
tf.config.optimizer.set_jit(True)

In [5]:
# the data, split between train and test sets
dataset = fashion_mnist
dataset = mnist


(x_train, y_train), (x_test, y_test) = dataset.load_data()

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train = x_train.astype(np.float32)/255
x_test = x_test.astype(np.float32)/255
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


In [6]:
def gen_model(activation="relu"):
  s = tf.keras.Input(shape=(28, 28, 1))
  h = tf.keras.layers.Flatten()(s)
  h = tf.keras.layers.Dense(128, activation=activation)(h)
  out = tf.keras.layers.Dense(10)(h)
  return tf.keras.Model(inputs=s, outputs=out)


def compile_model(model, lr=0.001):
  model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=keras.optimizers.Adam(learning_rate=lr),
                metrics=['accuracy'])
  

early_stop = tf.keras.callbacks.EarlyStopping(
      monitor='val_accuracy',
      patience=10,
      restore_best_weights=True,
)

In [7]:
%%time
with strategy.scope():
  model = gen_model()
  compile_model(model)
  model.fit(x_train, y_train,
            batch_size=128,
            epochs=100,
            verbose=1,
            callbacks=[early_stop],
            validation_data=(x_test, y_test),
            validation_batch_size=2048)


Epoch 1/100
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


  1/469 [..............................] - ETA: 1s - loss: 2.4464 - accuracy: 0.1250WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0036s vs `on_train_batch_end` time: 0.0154s). Check your callbacks.


469/469 [==============================] - ETA: 0s - loss: 0.3610 - accuracy: 0.9005WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0016s vs `on_test_batch_end` time: 0.0154s). Check your callbacks.


469/469 [==============================] - 11s 23ms/step - loss: 0.3610 - accuracy: 0.9005 - val_loss: 0.1913 - val_accuracy: 0.9444
Epoch 2/100
469/469 [==============================] - 8s 18ms/step - loss: 0.1635 - accuracy: 0.9525 - val_loss: 0.1402 - val_accuracy: 0.9568
Epoch 3/100
469/469 [==============================] - 8s 17ms/step - loss: 0.1181 - accuracy: 0.9659 - val_loss: 0.1131 - val_accuracy: 0.9674
Epoch 4/100
469/469 [==============================] - 8s 17ms/step - loss: 0.0919 - accuracy: 0.9735 - val_loss: 0.0957 - val_accuracy: 0.9705
Epoch 5/100
469/469 [==============================] - 8s 17ms/step - loss: 0.0750 - accuracy: 0.9782 - val_loss: 0.0915 - val_accuracy: 0.9717
Epoch 6/100
469/469 [==============================] - 8s 17ms/step - loss: 0.0622 - accuracy: 0.9819 - val_loss: 0.0829 - val_accuracy: 0.9745
Epoch 7/100
469/469 [==============================] - 9s 18ms/step - loss: 0.0514 - accuracy: 0.9852 - val_loss: 0.0803 - val_accuracy: 0.9738
Epo

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.07568933814764023
Test accuracy: 0.9803999662399292
